In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.callbacks import EarlyStopping

In [ ]:
# Load the training dataset
train_data = pd.read_csv('training.csv')

# Load the validation dataset
val_data = pd.read_csv('validation.csv')

# Load the test dataset
test_data = pd.read_csv('test.csv')

In [ ]:
# Drop the 'Run' and 'Event' columns from the training dataset
train_data = train_data.drop(['Run', 'Event'], axis=1)

# Drop the 'Run' and 'Event' columns from the validation dataset
val_data = val_data.drop(['Run', 'Event'], axis=1)

# Drop the 'Run' and 'Event' columns from the test dataset
test_data = test_data.drop(['Run', 'Event'], axis=1)

In [ ]:
# Extract the 'M' column as the target variable for training
y_train = train_data.pop('M')

# Extract the 'M' column as the target variable for validation
y_val = val_data.pop('M')

# Extract the 'M' column as the target variable for testing and remove it from test_data
y_test = test_data.pop('M')

# Convert the remaining data to feature sets
X_train = train_data.values
X_val = val_data.values
X_test = test_data.values

In [ ]:
input_shape = X_train.shape[1]  # Get the number of features (columns) in X_train

In [ ]:
#Define the model architecture
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(input_shape,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])

# Define the model architecture
model = keras.Sequential([
    Bidirectional(LSTM(64, activation='relu'), input_shape=(input_shape, 1)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=16, callbacks=[early_stopping])

# Step 7: Evaluate the model
y_pred = model.predict(X_test)

In [ ]:
# Calculate the MAD
median = np.median(y_train)
mad = np.median(np.abs(y_train - median))

print("Median Absolute Deviation (MAD):", mad)

In [ ]:
# To Calculate how many predictions fall within the range of ActualValue+-MAD*0.2
count_within_range = 0

# Print actual vs predicted values and count predictions within range
for i in range(len(y_pred)):
    print(f"Actual: {y_test[i]}, Predicted: {y_pred[i]}")
    if y_test[i] - 1 <= y_pred[i] <= y_test[i] + 1:
        count_within_range += 1

print("Count within range:", count_within_range/len(y_test))